In [ ]:
# NOTE: This code was used for experiments as-is. the code was ran in Enviornment 1: Kaggle as described in the paper
# Naming and structure may not follow programming best practices.
# Focus is on reproducibility.
#This code was developed for internal experimentation and contains hardcoded values for various test cases.
#It was not refactored for modularity, but the logic matches the experiments reported in the paper.


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import random
import os
import urllib.request
import zipfile
from PIL import Image

def set_all_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    print(seed)
#######################################################
#######################################################
#######################################################
#set_all_seeds(40)
#set_all_seeds(41)
set_all_seeds(42)
#set_all_seeds(43)
#set_all_seeds(44)
#set_all_seeds(45)
#set_all_seeds(46)
#set_all_seeds(47)
#set_all_seeds(48)
#set_all_seeds(49)
#######################################################
#######################################################
#######################################################

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TinyImageNet(torch.utils.data.Dataset):
    def __init__(self, root, split='train', transform=None, download=False):
        self.root = root
        self.split = split
        self.transform = transform
        
        if download:
            self.download()
        
        self.data_dir = os.path.join(root, 'tiny-imagenet-200')
        self.split_dir = os.path.join(self.data_dir, split)
        
        self.samples = []
        self.targets = []
        self.classes = []
        
        if split == 'train':
            self._load_train_data()
        else:
            self._load_val_data()
    
    def download(self):
        if not os.path.exists(os.path.join(self.root, 'tiny-imagenet-200')):
            # Create data directory if it doesn't exist
            os.makedirs(self.root, exist_ok=True)
            print("Downloading Tiny ImageNet...")
            url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
            zip_path = os.path.join(self.root, "tiny-imagenet-200.zip")
            urllib.request.urlretrieve(url, zip_path)
            
            print("Extracting...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(self.root)
            os.remove(zip_path)
    
    def _load_train_data(self):
        class_dirs = sorted(os.listdir(self.split_dir))
        self.classes = class_dirs
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_dirs)}
        
        for class_name in class_dirs:
            class_dir = os.path.join(self.split_dir, class_name, 'images')
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith('.JPEG'):
                        self.samples.append(os.path.join(class_dir, img_name))
                        self.targets.append(self.class_to_idx[class_name])
    
    def _load_val_data(self):
        # Load class names from train directory
        train_dir = os.path.join(self.data_dir, 'train')
        self.classes = sorted(os.listdir(train_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Read validation annotations
        val_annotations = os.path.join(self.split_dir, 'val_annotations.txt')
        with open(val_annotations, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                img_name, class_name = parts[0], parts[1]
                self.samples.append(os.path.join(self.split_dir, 'images', img_name))
                self.targets.append(self.class_to_idx[class_name])
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path = self.samples[idx]
        target = self.targets[idx]
        
        with Image.open(img_path) as img:
            img = img.convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        return img, target

# Define transforms for Tiny ImageNet (64x64 images)
transform = transforms.Compose([
    transforms.RandomCrop(64, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformtest = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_data = TinyImageNet(root='./data', split='train', transform=transform, download=True)
test_data = TinyImageNet(root='./data', split='val', transform=transformtest, download=False)

# Create DataLoaders for batching with increased batch size for efficiency
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# Example: Accessing the dataset
for images, labels in train_loader:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')
    print(f'Number of classes: {len(train_data.classes)}')
    break

import torch.nn.functional as F
class idle(nn.Module):
    def forward(self, x):
         #return x * torch.sigmoid(x)  #Swish
         #return 1.25*x * torch.sigmoid(x) #ESwish(UP)
         #return x*(torch.sigmoid(x)+0.125*torch.exp(-0.5*x**2))  #SwishPlus(UP)
         #return x * torch.tanh(F.softplus(x))   #Mish
         #return x * torch.tanh(F.softplus(0.9454113159514*x)/0.9454113159514)  #PMish(UP)
         #return x * torch.tanh(F.softplus(x)) +0.025*x*torch.exp(-0.5*x**2)  #MishPlus(UP)
         #return torch.relu(x) #ReLU
         #return F.gelu(x, approximate='tanh') #GeLU 
         return F.gelu(x, approximate='tanh') +0.125 * x * torch.exp(-0.5 * x**2) #GeLUPlus   
# Define ResNet model for Tiny ImageNet
class CustomResNet18(nn.Module):
    def __init__(self, num_classes=200, pretrained=False):
        super(CustomResNet18, self).__init__()
        self.resnet = resnet34(pretrained=pretrained)
        # Keep conv1 as is for 64x64 images (works well with 7x7 kernel)
        if pretrained:
            for param in self.resnet.parameters():
                param.requires_grad = False
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)
        self.replace_activations(self.resnet)

    def replace_activations(self, module):
        """
        Recursively replace all ReLU activations in the model,
        including those inside Sequential blocks
        """
        for name, child in module.named_children():
            if isinstance(child, nn.ReLU):
                # Direct replacement of ReLU modules
                new_activation = idle()
                if isinstance(module, nn.Sequential):
                    # For Sequential containers, we need to maintain the order
                    module[int(name)] = new_activation
                else:
                    setattr(module, name, new_activation)
            elif len(list(child.children())) > 0:
                # If module has children, recurse into them
                self.replace_activations(child)

    def forward(self, x):
        return self.resnet(x)

# Verification function to check if replacement worked
def verify_activation_replacement(model):
    """
    Verify that all ReLU activations have been replaced with Idle
    """
    def check_module(module):
        relu_count = 0
        idle_count = 0
        for child in module.modules():
            if isinstance(child, nn.ReLU):
                relu_count += 1
            if isinstance(child, idle):
                idle_count += 1
        return relu_count, idle_count

    relu_count, idle_count = check_module(model_idle)
    print(f"Found {relu_count} ReLU activations and {idle_count} Idle activations")
    assert relu_count == 0, "Some ReLU activations were not replaced!"
    return idle_count > 0

def calculate_batch_accuracy(outputs, labels):
    """Calculate accuracy for a single batch while in training mode"""
    _, predicted = torch.max(outputs, 1)
    return (predicted == labels).sum().item() / labels.size(0)

def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Training loop with proper metrics tracking
model_idle = CustomResNet18().to(device)
optimizer = optim.SGD(model_idle.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                    step_size=8,    # Decay LR every 8 epochs (more frequent for faster convergence)
                                    gamma=0.1)
criterion = nn.CrossEntropyLoss()
verify_activation_replacement(model_idle)
train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []
num_epochs=25  # Reduced epochs for time efficiency
for epoch in range(num_epochs):
    # Training phase
    model_idle.train()
    running_train_loss = 0.0
    running_train_acc = 0.0
    num_train_batches = 0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} (Training)'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_idle(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Calculate batch accuracy while in training mode
        batch_acc = calculate_batch_accuracy(outputs, labels)

        running_train_loss += loss.item()
        running_train_acc += batch_acc
        num_train_batches += 1

    scheduler.step()
    avg_train_loss = running_train_loss / num_train_batches
    avg_train_acc = running_train_acc / num_train_batches

    # Testing phase
    model_idle.eval()
    running_test_loss = 0.0
    running_test_acc = 0.0
    num_test_batches = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f'Epoch {epoch + 1}/{num_epochs} (Testing)'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_idle(inputs)
            loss = criterion(outputs, labels)

            batch_acc = calculate_batch_accuracy(outputs, labels)

            running_test_loss += loss.item()
            running_test_acc += batch_acc
            num_test_batches += 1

    avg_test_loss = running_test_loss / num_test_batches
    avg_test_acc = running_test_acc / num_test_batches

    # Store metrics
    train_accuracies.append(avg_train_acc)
    test_accuracies.append(avg_test_acc)
    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Training Loss: {avg_train_loss:.4f}, Training Accuracy: {avg_train_acc:.4f}')
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_acc:.4f}')